In [5]:
import csv
import re
import sys
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.special import entr
from scipy.stats import norm

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

from nltk.corpus import stopwords

In [6]:
# Needed functions for processing the data.

# Function to clean text data.
def clean_review(text):
    # Strip HTML tags
    text = re.sub('<[^<]+?>', ' ', text)
 
    # Strip escaped quotes
    text = text.replace('\\"', '')
 
    # Strip quotes
    text = text.replace('"', '')
    
    # Strip @
    text = text.replace('@', '')
 
    return text

# Puts words into a onehot embedding for the ML models ... I think.
def to_sequence(tokenizer, preprocessor, index, text):
    words = tokenizer(preprocessor(text))
    indexes = [index[word] for word in words if word in index]
    return indexes

# Shuffle the data function.
def shuffle(X, y):
    perm = np.random.permutation(len(X))
    X = X[perm]
    y = y[perm]
    return X, y

In [8]:
# Data processing for allImdb dataset.
# allImdb Cell 1/4

data_allImdb = []
reader_allImdb = csv.reader(open('allImdb.csv'), delimiter=',')
for row in reader_allImdb:
    data_allImdb.extend(row)

# Create empty lists to hold the sentences and sentiment data.
# These lists contain all the data. The data will still need to get shuffled and split
# into traing and test.
data_sentiment_allImdb = []
data_sentence_allImdb = []
for i in range(2, len(data_allImdb)):   # Start at index so dont record column labels.
    if (i % 2) == 0:
        data_sentiment_allImdb.extend(data_allImdb[i])
    else:
        data_sentence_allImdb.extend([data_allImdb[i]])

# Clean the sentence data i.e. removing html tags, ... etc.
for row in range(0, len(data_sentence_allImdb)):
    data_sentence_allImdb[row] = clean_review(data_sentence_allImdb[row])
  
# This next part has to be once for all the data ... Only execute the next few lines once!
vectorizer = CountVectorizer(binary=True, stop_words=stopwords.words('english'), 
                             lowercase=True, min_df=3, max_df=0.9, max_features=5000)

data_sentence_onehot_allImdb = vectorizer.fit_transform(data_sentence_allImdb)

word2idx = {word: idx for idx, word in enumerate(vectorizer.get_feature_names())}
tokenize = vectorizer.build_tokenizer()
preprocess = vectorizer.build_preprocessor()
N_FEATURES = len(vectorizer.get_feature_names())